<center>

**Vježba 5 - Razvijanje custom algoritama na scikit-learn osnovi**

</center>

U ovoj vježbi želimo razviti vlastiti estimator kao nadogranju na scikit-learn modul. U našem slučaju to će biti standardni k-means *clustering* algoritam (bez nadogradnji). Provjeriti će se kako se naš k-means algoritam uklapa u standardnu okolinu scikit-learn modula, a u skladu s time i provesti standardno k-struko unakrsno ispitivanje modela napravljenog na našem k-means algoritmu i postojećih clustering algoritama u scikit-learn modulu. Za učenje i testiranje clustering algoritama koristiti će se postojeći *iris* skup podataka.

[The Iris dataset](https://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html#the-iris-dataset)  
[Developing new estimators for scikit-learn](https://scikit-learn.org/stable/developers/develop.html)

Prvo učitamo sve potrebne python module, funkcije i klase. Klasu ```BaseEstimator``` koristimo kao osnovu za vlastiti k-means clustering algoritam. Implementiramo standardne metode tog bazičnog estimatora kao ```fit``` i ```predict```. Implementacija k-means algoritma je ponešto specifična jer se oslanja na pandas podatkovne strukture.

Vidimo da se bilježe centroidi clustera, te su to parametri koji bismo trebali spremiti u datoteku, ako se model želi spremiti i ponovno učitati kako bi se na njemu radio clustering nekih testnih podataka. Metode ```save``` i ```load``` se koriste kako bi unutarnje stanje našeg *clusterera* transformirale u JSON oblik, te zatim spremile u *custom* datoteku.

In [ ]:
from sklearn.base import BaseEstimator
from sklearn.datasets import load_iris
import pandas as pd
import json

class MyKMeans(BaseEstimator):
    def __init__(self, k: int):
        self.k = k
        self.clusters = {}
        self._estimator_type = "clusterer"

    def fit(self, X, y):
        s = 0
        if len(self.clusters) == 0:
            self.clusters["centroids"] = pd.DataFrame(X[:self.k])
            self.clusters["centroids"].reset_index(drop = True)
            self.clusters["N"] = [1] * self.k
            s = self.k
        for (i, row) in X[s:].iterrows():
            mind = None
            curr = -1
            for j in range(self.k):
                cls = self.clusters["centroids"].iloc[j]
                w = row-cls
                d = np.linalg.norm(w.values)
                if mind is None or mind > d:
                    mind, curr = d, j
            N = self.clusters["N"][curr]
            cennew = (self.clusters["centroids"].iloc[curr] * N + row) / (N + 1)
            self.clusters["N"][curr] = N + 1
            self.clusters["centroids"].iloc[curr] = cennew

    def predict(self, X):
        clus = []
        for (i, row) in X.iterrows():
            mind = None
            curr = -1
            for j in range(self.k):
                cls = self.clusters["centroids"].iloc[j]
                w = row-cls
                d = np.linalg.norm(w.values)
                if mind is None or mind > d:
                    mind, curr = d, j
            clus.append(curr)
        return clus

    def get_params(self, deep=True):
        return {"k": self.k}

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self

    def save(self, filename: str):
        with open(filename, 'w') as fp:
            json.dump({"k": self.k, "N": self.clusters["N"], "centroids": self.clusters["centroids"].to_json()}, fp)

    def load(self, filename: str):
        with open(filename, 'r') as fp:
            s = json.load(fp)
            self.k = s["k"]
            self.clusters = {}
            self.clusters["N"] = s["N"]
            self.clusters["centroids"] = pd.read_json(s["centroids"])

Prvo učitamo dodatne module, funkcije i klase za dodatni rad.

Zatim učitamo *iris* skup podataka razdvajajući labelu kao klasu cvijeta i njegove značajke. Zatim pogledamo značajke koje smo učitali.

Ovdje radimo k-struko unakrsno testiranje. Skup podataka dijelimo na tri dijela. Sva testiranja ponavljamo dva puta. Prije razdvajanja na dijelove radimo miješanje podataka o cvijetovima (ponovite gradivo o k-strukom unakrsnom testiranju iz predmeta Strojno Učenje).

Uzimamo tri clustering algoritma za testiranje:
- ```KMeans``` - Postojeći k-means++ algoritam iz scikit-learn modula.
- ```GaussianMixture``` - Statistički clustering algoritam koji koristi standardnu razdiobu za formiranje clustera.
- ```MyKMeans``` - Naša implementacija standardnog k-means clustering algoritma.

[KMeans](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html)  
[GaussianMixture](https://scikit-learn.org/stable/modules/generated/sklearn.mixture.GaussianMixture.html)

Izvodimo k-struko unakrsno testiranje na sva tri algoritma. Zatim ispisujemo *Adjusted Rand Index* (ARI) kao ocjenu modela za specifičnu za clustering algoritme.

Najbolji rezultat daje statistički clustering algoritam ```GaussianMixture```. Razlog tome je prirodna razdioba prisutna u fizičkim podacima o mjerenom cvijeću.  
Naša implementacija daje najslabiji rezultat, no uzrok tome je što je scikit-learn *k-mean* clustering algoritam značajno unaprijeđena verzija.

Primijetite da svako testiranje daje ponešto drukčiji rezultat. Zašto?

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import LabelEncoder
from sklearn.datasets import load_iris
import numpy as np

(iris_X, iris_y) = load_iris(return_X_y = True, as_frame = True)
display(iris_X)

kfr = RepeatedStratifiedKFold(n_splits = 3, n_repeats = 2, random_state = 1)

kmclus = KMeans(n_clusters = 3, n_init = 'auto')
gm = GaussianMixture(n_components = 3)
mkmclus = MyKMeans(3)

score_km = cross_val_score(kmclus, iris_X, iris_y, scoring = 'adjusted_rand_score', cv = kfr)
print(f'k-means ARI {np.mean(score_km)}')
score_gm = cross_val_score(gm, iris_X, iris_y, scoring = 'adjusted_rand_score', cv = kfr)
print(f'GaussianMixture ARI {np.mean(score_gm)}')
score_mkm = cross_val_score(mkmclus, iris_X, iris_y, scoring = 'adjusted_rand_score', cv = kfr)
print(f'My k-means ARI {np.mean(score_mkm)}')

Modul scikit-learn omogućava i ručni prolazak kroz k-struko unakrsno testiranje. To se postiže iterativnom petljom po objektu k-strukog unakrsnog testiranja. Pozivom metode ```split(X, y)``` dobivamo razdvajanje na podskup za učenje i testni podskup, a koji ovise o parametru ```n_splits``` u konstruktoru objekta k-strukog unakrsnog testiranja - u našem slučaju je to 3.

U svakom prolazu naučeni model spremimo u datoteku, pa ju učitamo, te na toj učitanoj instanci modela radimo testiranje. OČEKUJEMO da će rezultati biti isti kao i kod automatskog testiranja od strane scikit-learn modula.

In [ ]:
from sklearn.metrics import adjusted_rand_score

arim = []
for fold,(train,test) in enumerate(kfr.split(iris_X, iris_y)):
    mkmclus = MyKMeans(3)
    mkmclus.fit(iris_X.iloc[train], iris_y.iloc[train])    
    mkmclus.save('iris.aris')

    mkmclus2 = MyKMeans(3)
    mkmclus2.load('iris.aris')
    y = mkmclus2.predict(iris_X.iloc[test])
    ari = adjusted_rand_score(y, iris_y.iloc[test])
    arim.append(ari)

print(np.mean(arim))